# Tutorial: Examining Protected Areas and Their Benthic and Geomorphic Distribution

### Purpose
In this tutorial, we'll walk through using the [World Database on Protected Areas](https://www.protectedplanet.net/en/thematic-areas/wdpa) (WDPA) data and the Allen Coral Atlas (ACA) to do the following:

- determine the amount of benthic and geomorphic regions protected regions for Tonga

- examine whether sustainability goals are met based off of [Aichi Target 11](https://www.cbd.int/aichi-targets/target/11)

- find areas of interest suitable for protection status and policies

### Setup
We recommend using a Conda environment with the packages listed below. Instructions to add a Conda environment to a jupyter notebook can be found [here](https://medium.com/@nrk25693/how-to-add-your-conda-environment-to-your-jupyter-notebook-in-just-4-steps-abeab8b8d084), and Conda documentation can be found [here](https://docs.conda.io/en/latest/).
 
### Contents
 
 1. Obtaining Data from WDPA
 2. Visualizing Protected Regions
 3. Determine if Sustainability Goals are Met
 4. Find Areas of Interest Suitable for Protection Status and Policies

 
### Considerations
 - for ipyLeaflet maps to display, you may need to locally run the following commands:
 
        conda install -n your-environment-name -c conda-forge widgetsnbextension
        conda install -n your-environment-name -c conda-forge ipywidgets
 
 

### Packages to Import

In [132]:
from ipyleaflet import (Map, GeoData, basemaps, WidgetControl, GeoJSON,
    LayersControl, Icon, Marker,basemap_to_tiles, Choropleth,
    MarkerCluster, Heatmap,SearchControl, 
    FullScreenControl)
from ipywidgets import Text, HTML
from branca.colormap import linear

import geopandas as gpd
import json
import requests

from osgeo import ogr

from owslib.wfs import WebFeatureService
from owslib.wms import WebMapService

## 1. Obtaining data from WDPA
We'll be looking at Tonga again for protected areas. The data can be obtained from [here](https://www.protectedplanet.net/country/TON).

We'll be using the shapefiles which is listed as 'SHP' on the WDPA site.
When downloaded, the Tonga shapefiles have three parts and are located in the following folders: WDPA_WDOECM_TON_shp0, WDPA_WDOECM_TON_shp1, WDPA_WDOECM_TON_shp2.
Using geopandas, we'll work with the shapefiles as geodataframes for easier data analysis.

Let's start off by opening our downloaded shapefiles as geodataframes for all of the protected areas for Tonga (Note: file path needs to be changed to local location of downloaded Tonga shapefiles):

In [133]:
tonga1 = gpd.read_file('/Users/kirkl/workspace/aca-wms/WDPA_WDOECM_Feb2021_Public_TON_shp/WDPA_WDOECM_Feb2021_Public_TON_shp_0/WDPA_WDOECM_Feb2021_Public_TON_shp-polygons.shp')

In [134]:
tonga2 = gpd.read_file('/Users/kirkl/workspace/aca-wms/WDPA_WDOECM_Feb2021_Public_TON_shp/WDPA_WDOECM_Feb2021_Public_TON_shp_1/WDPA_WDOECM_Feb2021_Public_TON_shp-polygons.shp')

In [135]:
tonga3 = gpd.read_file('/Users/kirkl/workspace/aca-wms/WDPA_WDOECM_Feb2021_Public_TON_shp/WDPA_WDOECM_Feb2021_Public_TON_shp_2/WDPA_WDOECM_Feb2021_Public_TON_shp-polygons.shp')

Here, we have combined all of the shapefiles into one dataframe, `tonga`.

In [136]:
tonga_protected = tonga1.append(tonga2)
tonga_protected = tonga_protected.append(tonga3)

Now let's take a look at the data! The main pieces of data we're interested in are the protected area names, designation, managing authority, and the geometry of the area.

In [137]:
tonga_protected

,WDPAID,WDPA_PID,PA_DEF,NAME,ORIG_NAME,DESIG,DESIG_ENG,DESIG_TYPE,IUCN_CAT,INT_CRIT,...,MANG_AUTH,MANG_PLAN,VERIF,METADATAID,SUB_LOC,PARENT_ISO,ISO3,SUPP_INFO,CONS_OBJ,geometry
0,4235.0,4235,1,Mui Hopo Hoponga Coastal Reserve,Mui Hopo Hoponga Coastal Reserve,Reserve,Reserve,National,V,Not Applicable,...,Not Reported,Not Reported,State Verified,1942,Not Reported,TON,TON,Not Applicable,Not Applicable,"POLYGON ((-175.03242 -21.16160, -175.03402 -21..."
1,4236.0,4236,1,Monuafe Island Park and Reef,Monuafe Island Park and Reef,Reserve,Reserve,National,IV,Not Applicable,...,"Ministry of Lands, Survey & Natural Resources",Not Reported,State Verified,1942,Not Reported,TON,TON,Not Applicable,Not Applicable,"POLYGON ((-175.13777 -21.10335, -175.13841 -21..."
2,4237.0,4237,1,Malinoa Island Park and Reef,Malinoa Island Park and Reef,Reserve,Reserve,National,IV,Not Applicable,...,"Ministry of Lands, Survey & Natural Resources",Not Reported,State Verified,1942,Not Reported,TON,TON,Not Applicable,Not Applicable,"POLYGON ((-175.13199 -21.02851, -175.12886 -21..."
3,4238.0,4238,1,Pangaimotu Reef,Pangaimotu Reef,Reserve,Reserve,National,IV,Not Applicable,...,"Ministry of Lands, Survey & Natural Resources",Not Reported,State Verified,1942,Not Reported,TON,TON,Not Applicable,Not Applicable,"POLYGON ((-175.15355 -21.12127, -175.15728 -21..."
4,4239.0,4239,1,Ha'atafu Beach,Ha'atafu Beach,Reserve,Reserve,National,IV,Not Applicable,...,"Ministry of Lands, Survey & Natural Resources",Not Reported,State Verified,1942,Not Reported,TON,TON,Not Applicable,Not Applicable,"POLYGON ((-175.32775 -21.05188, -175.30481 -21..."
5,4240.0,4240,1,Hakaumama'o Reef,Hakaumama'o Reef,Reserve,Reserve,National,IV,Not Applicable,...,"Ministry of Lands, Survey & Natural Resources",Not Reported,State Verified,1942,Not Reported,TON,TON,Not Applicable,Not Applicable,"POLYGON ((-175.20427 -21.00507, -175.20661 -21..."
6,4241.0,4241,1,Fanga'uta and Fanga Kakau Lagoons,Fanga'uta and Fanga Kakau Lagoons,Marine Reserve,Marine Reserve,National,VI,Not Applicable,...,"Fisheries Division, Ministry of Lands, Survey ...",Not Reported,State Verified,1942,Not Reported,TON,TON,Not Applicable,Not Applicable,"POLYGON ((-175.15246 -21.13983, -175.15165 -21..."
7,18282.0,18282,1,Mounu Reef,Mounu Reef,Sanctuary,Sanctuary,National,Not Reported,Not Applicable,...,Not Reported,Not Reported,State Verified,1942,Not Reported,TON,TON,Not Applicable,Not Applicable,"POLYGON ((-175.18867 -21.12291, -175.18858 -21..."
8,28376.0,28376,1,'Eua National Park,'Eua,National Park,National Park,National,II,Not Applicable,...,Not Reported,Not Reported,State Verified,1942,Not Reported,TON,TON,Not Applicable,Not Applicable,"POLYGON ((-174.91745 -21.36994, -174.91745 -21..."
9,315554.0,315554,1,Falevai,Falevai SMA,Special Management Area,Special Management Area,National,VI,Not Applicable,...,Not Reported,Not Reported,State Verified,1942,Not Reported,TON,TON,Not Applicable,Not Applicable,"POLYGON ((-174.02851 -18.69925, -174.02846 -18..."


# 2. Visualizing Protected Regions
For visualization, we'll be using IPyLeaflet. Further details on IPyLeaflet library can be found in its official documentation [here](https://ipyleaflet.readthedocs.io/en/latest/) and in this tutorial [here](https://towardsdatascience.com/ipyleaflet-interactive-mapping-in-jupyter-notebook-994f19611e79).

We'll first create a basemap zoomed in with Tonga as the center.

In [138]:
center = [-21.248,-175.188]
zoom = 7


tonga_map = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=center, zoom=zoom)
tonga_map

Map(center=[-21.248, -175.188], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

Next, we'll add a layer which displays all the protected areas from the geometries in our `tonga_protected` dataframe.

In [139]:
protected_areas = GeoData(geo_dataframe = tonga_protected,
                  style={'color': 'black', 
                         'fillColor': '#E0D071',
                         'opacity':0.03, 
                         'weight':1.9, 'dashArray': '2', 
                         'fillOpacity':0.6},
                  hover_style={'fillColor': '#b08a3e', 
                               'fillOpacity': 0.8},
                  name = 'Tonga1')
tonga_map.add_layer(protected_areas)
tonga_map

Map(bottom=18564.0, center=[-21.248, -175.188], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zo…

We'll now add a guide which will display the name of the protected region, designation, and managing authority when hovering over a particular protected area.

In [140]:
html = HTML('''Hover Over Protected Areas''')
html.layout.margin = '0px 20px 20px 20px'
control = WidgetControl(widget=html, position='topright')
tonga_map.add_control(control)

def update_html(feature, **kwargs):
    html.value = '''
    <h3>{}</h3>
    <h4>Designation type: {}</h4>
    <h4>Managing authority: {}</h4>
    '''.format(feature['properties']['ORIG_NAME'],
              feature['properties']['DESIG'],
              feature['properties']['MANG_AUTH'])

protected_areas.on_hover(update_html)
tonga_map

Map(bottom=18564.0, center=[-21.248, -175.188], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zo…

# 3. Determine if Sustainability Goals are Met

Using the [Aichi Target 11](https://www.cbd.int/aichi-targets/target/11) as a guideline, we'll see if at least 10% of Tongan marine areas are marked as protected.
We shall first get the benthic and geomorphic classes within the EEZ of Tonga. The EEZ is obtained from MarineRegions and the benthic and geomorphic cover will be from the Allen Coral Atlas.

We'll repeat steps from our tutorial that obtains benthic and geomorphic cover from a given EEZ:
First, we'll create a dictionary with the keys being all the areas within the ACA and the values being the associated Marine Regions ID. We'll need the Marine Regions ID in the second section in order to obtain the EEZ of a specific country/area. To do so, we'll grab the Marine Regions EEZ map and restrict the output with a bounding box that is representative of the ACA area.

Note that this request can take up to 3 minutes.

In [141]:
wfs = WebFeatureService(url='https://geo.vliz.be/geoserver/MarineRegions/wfs', version='1.1.0')
response = wfs.getfeature(typename='MarineRegions:eez', 
                          bbox=(-180,-23.5,180,20), 
                          srsname='urn:x-ogc:def:crs:EPSG:4326', 
                          outputFormat='application/json')
eez_data = json.load(response)

Here, we create a dictionary and ensure that disputed and/or shared territories are included with the following filtering parameters.

In [142]:
mrgid_dict = {}
for country in eez_data['features']:
    if country['properties']['pol_type'] == '200NM':
        mrgid_dict[country['properties']['territory1']] = country['properties']['mrgid']
    else:
        mrgid_dict[country['properties']['geoname']] = country['properties']['mrgid']

We'll also use our `get_eez_map` function from the first tutorial to get Tonga's EEZ again.

In [143]:
def get_eez_map(country_mrgid):
    '''Given a country mrgid(Marine region country code), returns a geojson of a country\'s EEZ'''
    url = 'https://geo.vliz.be/geoserver/MarineRegions/wfs?service=WFS&version=1.1.0&request=GetFeature&typeName=eez&cql_filter=mrgid=' + str(country_mrgid) + '&outputformat=application/json'
    response = requests.get(url)
    eez = response.json()
    return eez

In [144]:
tonga_eez_json = get_eez_map(mrgid_dict['Tonga'])

We'll now take the geojson of Tonga's EEZ and transform it to a geodataframe.

In [145]:
tonga_eez = gpd.GeoDataFrame.from_features(tonga_eez_json)

Next we will (1) make a bounding box, (2) get the benthic and geomorphic layers, and (3) obtain the intersection between the ACA and EEZ layers.
Here is a summary of the functions to achieve these steps:
- `make_bounding_box(geojson)`: Transforms a geojson to a bounding box
- `get_aca_layer(layer, bounding_box)`: Given a bounding box, a request is made and downloads either the benthic and geomorphic layers from the Allen Coral Atlas
- `eez_intersection(aca_layer,eez)`: Get the intersection between the Allen Coral Atlas layers and the EEZ geojson from Marine Regions

In [146]:
def make_bounding_box(geojson):
    '''From a geojson, creates a bounding box which is a list of 4 coordinates'''
    geom_text = json.dumps(geojson["features"][0]["geometry"])
    geom = ogr.CreateGeometryFromJson(geom_text)
    env = geom.GetEnvelope()

    MAX_LONG = 180.0
    MIN_LONG = -180.0
    MAX_LAT = 90.0
    MIN_LAT = -90.0

    return (max(min(MAX_LONG, env[0]), MIN_LONG),
            max(min(MAX_LAT, env[2]), MIN_LAT),
            max(min(MAX_LONG, env[1]), MIN_LONG),
            max(min(MAX_LAT, env[3]), MIN_LAT))

In [147]:
def get_aca_layer(layer, bounding_box):
    '''Gets either geomorphic or benthic layer in geojson from the ACA and returns is a geopandas GeoDataFrame'''
    # layer options: ['reef_polygons_benthic_expanded'] or ['reef_polygons_geomorphic_expanded']
    wms = WebMapService('https://allencoralatlas.org/geoserver/coral-atlas/wms', version='1.3.0')
    country_aca_layer = wms.getmap(layers=layer, 
                                   srs='EPSG:4326', 
                                   bbox=bounding_box, 
                                   size=(1000,1000),
                                   format='application/json;type=geojson')
    try:
        country_aca_layer_geojson = json.load(country_aca_layer)
        return gpd.GeoDataFrame.from_features(country_aca_layer_geojson)
    except ValueError:
        if country_aca_layer._response.status_code == 429:
            print('Too many requests have been made in a given amount of time, please wait another minute to send another request.')
        else:
            print('Error acquiring layer, response code: ' + country_aca_layer._response.status_code)         

Let's first get the bounding box.

In [148]:
tongabbox = make_bounding_box(tonga_eez_json)

We'll now take our `tongabbox` and obtain the benthic and geomorphic layers using our previously defined `get_aca_layer` function we defined earlier. Currently, to request the benthic layer, ['benthic_data_verbose'], is used and to request the geomorphic layer, ['geomorphic_data_verbose'], is used.

In [149]:
benthic_tonga = get_aca_layer(['benthic_data_verbose'], tongabbox)

In [150]:
geomorphic_tonga = get_aca_layer(['geomorphic_data_verbose'], tongabbox)

We'll now obtain the intersection between the ACA layers and the Marine Regions EEZ. The intersection will be a GeoDataFrame which will make data analysis easier.

In [151]:
benthic_tonga_eez = gpd.overlay(benthic_tonga,
                                tonga_eez,
                                how="intersection").set_crs('EPSG:4326')

In [152]:
geomorphic_tonga_eez = gpd.overlay(geomorphic_tonga,
                                   tonga_eez,
                                   how="intersection").set_crs('EPSG:4326')

We'll now determine the benthic and geomorphic cover within Tonga's current protected regions.

In [153]:
benthic_tonga_eez_protected = gpd.overlay(benthic_tonga_eez,
                                          tonga_protected,
                                          how="intersection").set_crs('EPSG:4326')

In [154]:
geomorphic_tonga_eez_protected = gpd.overlay(geomorphic_tonga_eez,
                                             tonga_protected,
                                             how="intersection").set_crs('EPSG:4326')

We can now use IPyLeaflet to visualize the benthic and geomorphic regions within the protected areas.

In [155]:
benthic = GeoData(geo_dataframe = benthic_tonga_eez_protected,
                  style={'color': 'orange', 
                         'fillColor': '#b0553e',
                         'opacity':0.03, 
                         'weight':1.9, 'dashArray': '2', 
                         'fillOpacity':0.8},
                  hover_style={'fillColor': '#b0553e', 
                               'fillOpacity': 0.9},
                  name = 'benthic')
tonga_map.add_layer(benthic)
tonga_map

Map(bottom=18564.0, center=[-21.248, -175.188], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zo…

In [156]:
geomorphic = GeoData(geo_dataframe = geomorphic_tonga_eez_protected,
                  style={'color': 'blue', 
                         'fillColor': '#7b57f0',
                         'opacity':0.03, 
                         'weight':1.9, 'dashArray': '2', 
                         'fillOpacity':0.8},
                  hover_style={'fillColor': '#7b57f0', 
                               'fillOpacity': 0.9},
                  name = 'geomorphic')
tonga_map.add_layer(geomorphic)
tonga_map

Map(bottom=147022.0703125, center=[-21.133183416835212, -175.03761291503906], controls=(ZoomControl(options=['…

Let's now calculate the numerical breakdown of the amount of benthic and geomorphic cover that is protected. We'll first reproject our data using the `reproject` function defined below. The Mollweide projection will be used which is an equal area projection. This will give a more accurate estimate of the protected areas in square meters.

In [157]:
def reproject(eez_geodataframe, projection_identifier):
    '''Reprojects a geodataframe to the user provided projection'''
    # e.g. Mollweide would be 'ESRI:54009'
    # set original CRS before reprojecting, in this case, the data obtained 
    # from the Allen Coral Atlas is in the EPSG:4326 format
    eez_geodataframe.crs = 'EPSG:4326'
    
    # adds a column, 'reprojected_area' that converts the 
    # current area to user provided projection
    eez_geodataframe['total_area'] = eez_geodataframe['geometry'].to_crs(projection_identifier).map(lambda p: p.area / 10**6)
    
    return eez_geodataframe

In [158]:
reprojected_tonga_benthic = reproject(benthic_tonga_eez, 'ESRI:54009')
reprojected_tonga_geomorphic = reproject(geomorphic_tonga_eez, 'ESRI:54009')

Using Pandas, we'll now query the `reprojected_tonga_benthic` and `reprojected_tonga_geomorphic` dataframes to calculate Tonga's percent distribution of subtypes for benthic and geomorphic alongside the percentage that is protected.

In [159]:
def calculate_percent_distribution(eez_geodataframe): 
    # generates a new dataframe containing the subtotal 
    # of the area of benthic/geomorphic classes 
    eez_geodataframe_subtotals = eez_geodataframe.groupby('class_name')['total_area'].sum().to_frame()

    return eez_geodataframe_subtotals

Let's calculate the benthic percentage first.

In [160]:
benthic_tonga_dist = calculate_percent_distribution(reprojected_tonga_benthic)
benthic_tonga_dist['protected_area'] = reprojected_tonga_benthic_protected.groupby('class_name')['total_area'].sum().to_frame()
benthic_tonga_dist['percent_protected'] = benthic_tonga_dist['protected_area']/benthic_tonga_dist['total_area'] * 100

NameError: name 'reprojected_tonga_benthic_protected' is not defined

Taking a closer look at the data, we see that for all 6 subtypes, it meets the Aichi 11 goal of having greater than 10% of the area preserved.

In [33]:
benthic_tonga_dist

,total_area,protected_area,percent_protected
class_name,,,
Coral/Algae,490.600313,490.600313,100.0
Microalgal Mats,63.815690,63.815690,100.0
Rock,109.303179,109.303179,100.0
Rubble,101.943349,101.943349,100.0
Sand,113.354787,113.354787,100.0
Seagrass,28.566253,28.566253,100.0


Now let's do the same for the geomorphic data.

In [34]:
geomorphic_tonga_dist = calculate_percent_distribution(reprojected_tonga_geomorphic)
geomorphic_tonga_dist['protected_area'] = reprojected_tonga_geomorphic_protected.groupby('class_name')['total_area'].sum().to_frame()
geomorphic_tonga_dist['percent_protected'] = geomorphic_tonga_dist['protected_area']/geomorphic_tonga_dist['total_area'] * 100

We see that the geeomorphic regions also have greater than 10% of the area preserved.

In [35]:
geomorphic_tonga_dist

,total_area,protected_area,percent_protected
class_name,,,
Back Reef Slope,95.545985,95.545985,100.0
Deep Lagoon,142.132211,142.132211,100.0
Inner Reef Flat,117.760453,117.760453,100.0
Outer Reef Flat,138.736016,138.736016,100.0
Plateau,107.088975,107.088975,100.0
Reef Crest,39.776844,39.776844,100.0
Reef Slope,311.145205,311.145205,100.0
Shallow Lagoon,87.278856,87.278856,100.0
Sheltered Reef Slope,67.412783,67.412783,100.0


# 4. Visualizing Unprotected Areas

In [281]:
benthic_tonga_eez.crs = 'EPSG:4326'

unprotected_benthic_tonga = gpd.overlay(benthic_tonga_eez, tonga_protected, how='difference')

In [280]:
geomorphic_tonga_eez.crs = 'EPSG:4326'

unprotected_geomorphic_tonga = gpd.overlay(geomorphic_tonga_eez, tonga_protected, how='difference')

In [288]:
center = [-21.248,-175.188]
zoom = 7


tonga_unprotected_areas_map = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=center, zoom=zoom)
tonga_unprotected_areas_map

Map(center=[-21.248, -175.188], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

In [289]:
unprotected_geomorphic = GeoData(geo_dataframe = unprotected_geomorphic_tonga,
                  style={'color': 'blue', 
                         'fillColor': '#10bda6',
                         'opacity':0.03, 
                         'weight':1.9, 'dashArray': '2', 
                         'fillOpacity':0.8},
                  hover_style={'fillColor': '#10bda6', 
                               'fillOpacity': 0.9},
                  name = 'unprotected_geomorphic')
tonga_unprotected_areas_map.add_layer(geomorphic)
tonga_unprotected_areas_map

Map(bottom=18564.0, center=[-21.248, -175.188], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zo…